In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .appName("Combustiveis Analysis")
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/22 10:40:10 WARN Utils: Your hostname, GTI-DADOS, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/22 10:40:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/22 10:40:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/22 10:40:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('encoding', 'ISO-8859-1')
    .csv('../data/raw/precos-gasolina-etanol-09.csv')
)

In [4]:
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [5]:
df.show()

+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+--------------------+--------------+---------+------------------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|         Complemento|        Bairro|      Cep|           Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+--------------------+--------------+---------+------------------+--------------+--------------+---------------+-----------------+--------+
|            SE|            SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|       306|                NULL|JARDIM ZULMIRA|18061-000|          GASOLINA|    01/09/2025|          5,59|           NULL|       R$ / litr

In [10]:
df_preco = (
    df
    .select("Estado - Sigla", "Produto", "Valor de Compra", "Valor de Venda", "Unidade de Medida")
)

In [12]:
df_preco.show(5)

+--------------+--------+---------------+--------------+-----------------+
|Estado - Sigla| Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+--------+---------------+--------------+-----------------+
|            SP|GASOLINA|           NULL|          5,59|       R$ / litro|
|            SP|  ETANOL|           NULL|          3,69|       R$ / litro|
|            AP|GASOLINA|           NULL|          5,95|       R$ / litro|
|            AP|GASOLINA|           NULL|          5,95|       R$ / litro|
|            AP|GASOLINA|           NULL|          5,95|       R$ / litro|
+--------------+--------+---------------+--------------+-----------------+
only showing top 5 rows


In [13]:
(
    df_preco
    .where(
        F.col("Valor de Compra").isNotNull()
    )
    .show()
)

+--------------+-------+---------------+--------------+-----------------+
|Estado - Sigla|Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+-------+---------------+--------------+-----------------+
+--------------+-------+---------------+--------------+-----------------+



In [26]:
df_preco = (
    df
    .select("Estado - Sigla", "Produto", "Valor de Venda", "Unidade de Medida")
    .withColumn(
        "Valor de Venda",
        F.regexp_replace(F.col("Valor de Venda"), ",", ".")
        .cast("float")
    )
)

In [29]:
(
    df_preco
    .groupBy(
        F.col("Estado - Sigla"),
        F.col("Produto"),
        F.col("Unidade de Medida")
    )
    .agg(
        F.min(F.col("Valor de Venda")).alias("Menor Preço"),
        F.max(F.col("Valor de Venda")).alias("Maior Preço"),
    )
    .withColumn(
        "Diferença",
        F.col("Maior Preço") - F.col("Menor Preço")
    )
    .orderBy("Diferença", ascending=False)
    .show()    
        
)

+--------------+------------------+-----------------+-----------+-----------+---------+
|Estado - Sigla|           Produto|Unidade de Medida|Menor Preço|Maior Preço|Diferença|
+--------------+------------------+-----------------+-----------+-----------+---------+
|            SP|GASOLINA ADITIVADA|       R$ / litro|       5.19|       9.59|      4.4|
|            SP|          GASOLINA|       R$ / litro|       4.99|       9.19|      4.2|
|            SP|            ETANOL|       R$ / litro|       3.19|       6.39|3.1999998|
|            BA|GASOLINA ADITIVADA|       R$ / litro|       5.44|       8.49|3.0499997|
|            RJ|          GASOLINA|       R$ / litro|       5.39|       7.79|      2.4|
|            RJ|GASOLINA ADITIVADA|       R$ / litro|       5.49|       7.79|2.3000002|
|            PE|          GASOLINA|       R$ / litro|       5.34|       7.59|     2.25|
|            PE|            ETANOL|       R$ / litro|       4.34|       6.49|2.1499996|
|            RS|          GASOLI